## Introduction
This notebook runs all the processing steps one by one for several models and renders the output. Each section is individually runnable after a kernel restart 

## Observations
* Symbolic tracing did not play well with any BERT model, because it creates proxies for mutually exclusive inputs to e.g. `DistilBertModel.forward`
  * This was fixed by making the `concrete_args` input to `fx.symbolic_trace` available to the `MAV` and `MavTracer` objects
  * For BERT models, `concrete_args={'inputs_embeds':None}` gets around this issue
* Still, most NLP models use proxy variables for control flow, which is not supported by `torch.fx`
  * Perhaps fixing more arguments via `concrete_args` could work around this. To be investigated.

## Wav2Vec

In [1]:
import sys
sys.path.append('..')
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import torch
from idlmav import MAV, plotly_renderer

model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model.eval()
inputs = torch.randn(1, 16000)
device = 'cpu'

mav = MAV(model, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

/home/dev/ai/idlmav/.venv/lib/python3.10/site-packages/transformers/configuration_utils.py:312: UserWarning:

Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.



Tracing failed with torch.fx.symbolic_trace: zeros() received an invalid combination of arguments - got (tuple, device=Attribute, dtype=Attribute), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)
 * (tuple of ints size, *, Tensor out = None, torch.dtype dtype = None, torch.layout layout = None, torch.device device = None, bool pin_memory = False, bool requires_grad = False)

Tracing with torch.compile


INFO:2025-02-22 12:15:28 28484:28484 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


## Whisper tiny (encoder)

In [ ]:
import sys
sys.path.append('..')
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from idlmav import MAV, plotly_renderer

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model.eval()
inputs = torch.randn((1,80,3000))
device = 'cpu'

mav = MAV(model.model.encoder, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


## Whisper tiny (decoder)

In [1]:
import sys
sys.path.append('..')
from transformers import WhisperProcessor, WhisperModel
import torch
from idlmav import MAV, plotly_renderer, MavOptions

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperModel.from_pretrained("openai/whisper-tiny")

# Encode
input_features = torch.randn(1, 80, 3000)  # Replace with actual features from processor.feature_extractor
encoder_outputs = model.encoder(input_features)
encoder_hidden_states = encoder_outputs.last_hidden_state  # (batch_size, seq_length, hidden_dim)

# Decode
decoder_input_ids = torch.tensor([[processor.tokenizer.bos_token_id]])
decoder_outputs = model.decoder(input_ids=decoder_input_ids, encoder_hidden_states=encoder_hidden_states)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [6]:
decoder_inputs = dict(input_ids=decoder_input_ids, 
                      encoder_hidden_states=encoder_hidden_states,
                      attention_mask=None,
                      head_mask=None,
                      cross_attn_head_mask=None,
                      past_key_values=None,
                      inputs_embeds=None,
                      position_ids=None,
                      use_cache=False,
                      output_attentions=None,
                      output_hidden_states=None,
                      return_dict=None,
                      cache_position=None)
mav = MAV(model.decoder, decoder_inputs, MavOptions(try_fx_first=False, keep_internal_nodes=False))
with plotly_renderer('notebook_connected'): mav.show_figure()

* decoder entry point in [modeling_whisper.py:1210](../.venv/lib/python3.10/site-packages/transformers/models/whisper/modeling_whisper.py:1210)
* decoder initially called from `detect_language` in [generation_whisper.py:1470](../.venv/lib/python3.10/site-packages/transformers/models/whisper/generation_whisper.py:1470)
* decoder eventually called from [utils.py:2255](../.venv/lib/python3.10/site-packages/transformers/generation/utils.py:2255) (`GeneratorMixin._sample` in `GenerationMode.GREEDY_SEARCH`)
* [utils.py:1813](../.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1813): `model_kwargs['past_key_values'] = EncoderDecoderCache(DynamicCache(), DynamicCache())`
* Arguments passed to decoder both times:
  * `input_ids`  `torch.int64` tensor of shape initially `[1,1]`, then `[1,4]` e.g. `[[50258, 50259, 50359, 50363]]`
  * `encoder_hidden_states`: `torch.float32` tensor of shape `([1, 1500, 384])`

## Whisper tiny (full model)

In [1]:
import sys
sys.path.append('..')
from transformers import WhisperProcessor, WhisperModel
import torch
from idlmav import MAV, plotly_renderer, MavOptions

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperModel.from_pretrained("openai/whisper-tiny")

# Encode
input_features = torch.randn(1, 80, 3000)  # Replace with actual features from processor.feature_extractor
encoder_outputs = model.encoder(input_features)
encoder_hidden_states = encoder_outputs.last_hidden_state  # (batch_size, seq_length, hidden_dim)

# Decode
decoder_input_ids = torch.tensor([[model.config.bos_token_id]])
decoder_outputs = model.decoder(input_ids=decoder_input_ids, encoder_hidden_states=encoder_hidden_states)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [2]:
input_dict = dict(input_features=input_features, 
                  decoder_input_ids=torch.Tensor([[model.config.decoder_start_token_id]]).to(torch.int64),
                  use_cache=False)
mav = MAV(model, input_dict, MavOptions(try_fx_first=False, keep_internal_nodes=False))
with plotly_renderer('notebook_connected'): mav.show_figure()

## Whisper tiny (full model including generator)

In [1]:
import sys
sys.path.append('..')
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from idlmav import MAV, plotly_renderer, MavOptions

# Load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

# Encode and decode
input_features = torch.randn(1, 80, 3000)  # Replace with actual features from processor.feature_extractor
input_dict = dict(input_features=input_features, 
                  decoder_input_ids=torch.Tensor([[model.config.decoder_start_token_id]]).to(torch.int64),
                  use_cache=False)
mav = MAV(model, input_dict, MavOptions(try_fx_first=False, keep_internal_nodes=False))
with plotly_renderer('notebook_connected'): mav.show_figure()

## Whisper tiny (full model including audio sample)

In [1]:
!wget -Otim.mp3 https://upload.wikimedia.org/wikipedia/commons/transcoded/1/16/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga.mp3

--2025-03-16 12:36:09--  https://upload.wikimedia.org/wikipedia/commons/transcoded/1/16/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga.mp3
Resolving upload.wikimedia.org (upload.wikimedia.org)... 185.15.58.240, 2a02:ec80:600:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|185.15.58.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1256392 (1.2M) [audio/mpeg]
Saving to: ‘tim.mp3’

tim.mp3             100%[===================>]   1.20M  1.15MB/s    in 1.0s    

2025-03-16 12:36:11 (1.15 MB/s) - ‘tim.mp3’ saved [1256392/1256392]



In [2]:
import sys
sys.path.append('..')
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch, torchaudio
import torchaudio.functional as F
import IPython.display as ipd
from idlmav import MAV, MavOptions, plotly_renderer

y1, sr1 =  torchaudio.load("tim.mp3")
sr = 16000
y = F.resample(y1, sr1, sr)
ipd.Audio(y[0], rate=sr)

In [3]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model.eval()

input_features = processor(y[0], sampling_rate=sr, return_tensors="pt").input_features 
predicted_ids = model.generate(input_features)
transcription = processor.batch_decode(predicted_ids)
transcription

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[" Well, it's not every placement for the web, the web continues. But when you think of the files on your computer, the documents, the email messages and letters and things, the things that you can put on the web now. But then there are data files on there like calendars and downloaded spreadsheets and things, which you can't really put on the web. Because if you put calendar on the web, you have to put it on a document. And with a computer, you can't do all the things you need to do with data, like with a calendar, you need to be able to look at it in a day view, month,"]

In [4]:
input_dict = dict(input_features=input_features, 
                  decoder_input_ids=torch.Tensor([[model.config.decoder_start_token_id]]).to(torch.int64),
                  use_cache=False)
mav = MAV(model, input_dict, MavOptions(try_fx_first=False, keep_internal_nodes=False))
with plotly_renderer('notebook_connected'): mav.show_figure()

## Audio Spectrogram Transformer (AST)
* TODO: Figure out required input size

In [1]:
import sys
sys.path.append('..')
from transformers import ASTConfig, ASTModel
import torch
from idlmav import MAV, plotly_renderer

config = ASTConfig()
model = ASTModel(config)
model.eval()
inputs = torch.randn(1, config.max_length, config.num_mel_bins)
device = 'cpu'

mav = MAV(model, inputs)
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


INFO:2025-02-22 12:41:14 38951:38951 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


## Seamless M4T
* TODO: Run on better hardware

In [ ]:
!wget -Otim.mp3 https://upload.wikimedia.org/wikipedia/commons/transcoded/1/16/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga.mp3

--2025-02-19 02:06:21--  https://upload.wikimedia.org/wikipedia/commons/transcoded/1/16/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga/Tim_Berners-Lee_-_Today_%28flac_-sample_s16_-f_ogg%29.oga.mp3
Resolving upload.wikimedia.org (upload.wikimedia.org)... 185.15.58.240, 2a02:ec80:600:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|185.15.58.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1256392 (1.2M) [audio/mpeg]
Saving to: ‘tim.mp3’

tim.mp3             100%[===================>]   1.20M  1.16MB/s    in 1.0s    

2025-02-19 02:06:23 (1.16 MB/s) - ‘tim.mp3’ saved [1256392/1256392]



In [ ]:
import torchaudio
from transformers import AutoProcessor, SeamlessM4TModel
processor = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-medium", sampling_rate=8_000)
model = SeamlessM4TModel.from_pretrained("facebook/hf-seamless-m4t-medium", sampling_rate=8_000)

# Read an audio file and resample:
audio, orig_freq =  torchaudio.load("tim.mp3")
audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=8_000) # must be a 16 kHz waveform array
audio_inputs = processor(audios=audio, return_tensors="pt")


It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [ ]:
# Convert audio to text
output_tokens = model.generate(**audio_inputs, tgt_lang="eng", generate_speech=False)
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
translated_text_from_audio


"But when you think about the files on your computer, the emails, the files you put on the web, but there are data files on the web, like calendars and downloads, which you can't put on the web, because if you put the calendar on the web, you have to do everything you need to do with the data you need to compare the other calendars at the same time."

In [ ]:
import sys
sys.path.append('..')
from idlmav import MAV, plotly_renderer

mav = MAV(model.speech_encoder, audio_inputs, device='cpu')
with plotly_renderer('notebook_connected'): mav.show_figure()

Tracing failed with torch.fx.symbolic_trace: symbolically traced variables cannot be used as inputs to control flow
Tracing with torch.compile


INFO:2025-02-19 02:24:18 174222:174222 init.cpp:181] If you see CUPTI_ERROR_INSUFFICIENT_PRIVILEGES, refer to https://developer.nvidia.com/nvidia-development-tools-solutions-err-nvgpuctrperm-cupti


: 

## Others under consideration
* [melodyflow-t24-30secs](https://huggingface.co/facebook/melodyflow-t24-30secs)
* [MagNET](https://huggingface.co/facebook/magnet-small-30secs)